Triangulation of a point from two images
----------------------------------------------------

__Inputs:__ 

$K_1$, $K_2$ - 3x3 camera intrinsic calibration/parameters matrix (depends on focal distance, resolution, optical center, alignment of lens)

$D_1$, $D_2$ - 1x5 camera disortion coefficients (assumes pinhole camera model)

$p_1$, $p_2$ - 1x2 tracking marker in image coordinates (unrectified/undisorted)

$R_2$ - 3x3 rotation matrix with respect to first camera

$T_2$ - 1x3 translation vector with respect to first camera

__Important intermediate variables:__

$P_n$ - 3x4 projection matrix of camera. Equals to [R_n | T_n] $ where | means horizontal stacking


Two possible scenarios:

1. If image is already undistorted, let $P_n = K_n \times [R_n | Tn]$ and pass image coordinates to triangulatePoints.
2. If image is not undistorted, call undistortPoints to get normalized point coordinates and let $P_n = [R_n | Tn]$ (so that triangulatePoints doesn't use calibration matrix more than once).

For better performance, it's better to find markers in raw image and undistort just those points instead of all image (scenario 2).



Sequence of steps for scenario 2:

1. undisortPoints is called to remove lens disortion with points in image coordinates.
2. $P_n = [R_n | T_n]
3. triangulatePoints(P_1, P_2, undistorted_p_1, undistorted_p_2) -> P_3D 1x4 matrix. To get world coordinates, divide by last element (Pt = P_3D / P_3D[3])




In [2]:
import numpy as np

Mat_K = np.matrix([[544.433304, 0, 314.924406], [0, 543.746983, 240.672811],[ 0, 0, 1]])


print "Intrinsic camera matrix"
print Mat_K

Mat_D = np.matrix([-0.443683, 0.195487, 0.000762, 0.000193, 0])

print "Disortion coefficients"

print Mat_D



Intrinsic camera matrix
[[ 544.433304    0.        314.924406]
 [   0.        543.746983  240.672811]
 [   0.          0.          1.      ]]
Disortion coefficients
[[ -4.43683000e-01   1.95487000e-01   7.62000000e-04   1.93000000e-04
    0.00000000e+00]]


In [3]:
import cv2

from numpy.linalg import inv

#p_1 = np.array([[[321.05,267.23]]], dtype=np.float32)

#p_2 = np.array([[[53.19, 261.80]]], dtype=np.float32)

p_1 = np.array([[[581.30,255.57]]], dtype=np.float32)

p_2 = np.array([[[334.49, 252.09]]], dtype=np.float32)

p_1_undistorted = cv2.undistortPoints(p_1, Mat_K, Mat_D)

p_2_undistorted = cv2.undistortPoints(p_2, Mat_K, Mat_D)

print p_1_undistorted

print p_2_undistorted


[[[ 0.55466002  0.0307961 ]]]
[[[ 0.0359632   0.02101111]]]


In [4]:
"""Construct projection matrices of both cameras"""

P_1 = np.hstack((np.eye(3), np.matrix([0, 0, 0]).T))

R12 = np.eye(3) # no rotation for cameras

T12 = np.matrix([-0.5, 0, 0])

P_2 = np.hstack((R12, T12.T))



print "P_1"

print P_1

print "P_2"
print P_2


P_1
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]]
P_2
[[ 1.   0.   0.  -0.5]
 [ 0.   1.   0.   0. ]
 [ 0.   0.   1.   0. ]]


In [5]:
import cv2


Pt_3D = cv2.triangulatePoints(P_1, P_2, np.array([0.5546600, 0.0307961]),np.array([0.0359632, 0.02101111]) )

print Pt_3D / Pt_3D[3]

[[ 0.53462042]
 [ 0.0249656 ]
 [ 0.96377878]
 [ 1.        ]]
